### TODO

- Correct it, it's finding local mimima
- Find the right way to turn the solution into integers

In [55]:
import numpy as np
import scipy.optimize as optimize

def init_simulation(num_nodes, 
                    num_days, 
                    icu_capacities = None, 
                    transport_capacities = None, 
                    ini_path = None, 
                    demand_min = 80, 
                    demand_max = 90,
                    icu_min = 10,
                    icu_max = 300,
                    transport_min = 20,
                    transport_max = 30):
    if ini_path:
        print("you can also upload an .ini file")
        raise NotImplemented
    else:
        demand = []
        for node in range(num_nodes):
            demand.append(list(np.random.randint(demand_min, demand_max, size = num_days)))
        
        icu_capacities = list(np.random.randint(icu_min, icu_max, size = num_nodes)) if not icu_capacities else icu_capacities
        transport_capacities = list(np.random.randint(transport_min, transport_max, size = num_nodes)) if not transport_capacities else transport_capacities        
    
    movements_orig = np.array([[[0]*num_nodes]*num_nodes]*num_days)
    return icu_capacities, transport_capacities, demand, movements_orig


def calculate_outgoing(array,day,node):
    return sum(array[day][node])


def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list

def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

def f_cons(node_id, day):
    return lambda x: transport_capacities[node_id] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),day ,node_id)

In [133]:
prob_death_transport = 0
num_days = 10
num_nodes = 4
node_capacities, transport_capacities, demands, movements_orig = init_simulation(num_nodes, 
                                                                                 num_days, 
                                                                                 icu_capacities=[30000, 20, 80, 1000],
                                                                                 transport_capacities=[50, 50, 50, 50], 
                                                                                 demand_min = 60, demand_max=100)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}
Initial guess (no one moves):
{movements_orig}""")

Randomly generated
10 days, 4 hospitals
ICU capacities of hospitals: [30000, 20, 80, 1000]
Daily transport capacities: [50, 50, 50, 50], 
Daily simulated demand (from uniform distrution): 
[[93, 67, 61, 68, 63, 77, 62, 64, 92, 68], [60, 98, 92, 99, 79, 79, 78, 93, 79, 70], [62, 81, 98, 68, 75, 98, 67, 65, 76, 89], [81, 94, 68, 62, 64, 87, 88, 61, 98, 73]]
Initial guess (no one moves):
[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [134]:
#flatten the movements array from 3D to 1D
movements = movements_orig.ravel()
#define constraints
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day)})

bounds_movement = generate_bounds()
#the actual
result = optimize.minimize(calc_total_deaths, 
                           movements, 
                           method='SLSQP', 
                           constraints=cons, 
                           bounds=bounds_movement)

In [135]:
np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))

array([[[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [40.,  0.,  0., 10.],
        [40.,  0.,  0., 10.],
        [29.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., 

In [136]:
calc_total_deaths(np.rint(result.x))

506.0

In [137]:
calc_total_deaths(movements_orig)

1506

In [124]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 376.0
Node: 0, Day: 1, Demand: 758.0
Node: 0, Day: 2, Demand: 1148.0
Node: 0, Day: 3, Demand: 1545.0
Node: 0, Day: 4, Demand: 1943.0
Node: 0, Day: 5, Demand: 2324.0
Node: 0, Day: 6, Demand: 2708.0
Node: 0, Day: 7, Demand: 3095.0
Node: 0, Day: 8, Demand: 3482.0
Node: 0, Day: 9, Demand: 3877.0
Node: 1, Day: 0, Demand: 0
Node: 1, Day: 1, Demand: 0
Node: 1, Day: 2, Demand: 0
Node: 1, Day: 3, Demand: 0
Node: 1, Day: 4, Demand: 0
Node: 1, Day: 5, Demand: 0
Node: 1, Day: 6, Demand: 0
Node: 1, Day: 7, Demand: 0
Node: 1, Day: 8, Demand: 0
Node: 1, Day: 9, Demand: 0
Node: 2, Day: 0, Demand: 0
Node: 2, Day: 1, Demand: 0
Node: 2, Day: 2, Demand: 0
Node: 2, Day: 3, Demand: 0
Node: 2, Day: 4, Demand: 0
Node: 2, Day: 5, Demand: 0
Node: 2, Day: 6, Demand: 0
Node: 2, Day: 7, Demand: 0
Node: 2, Day: 8, Demand: 0
Node: 2, Day: 9, Demand: 0


In [52]:
demand_day_node(np.rint(result.x),3,2)

IndexError: index 2 is out of bounds for axis 0 with size 2

# SANDBOX

In [53]:
prob_death_transport = 0
num_days = 1
num_nodes = 2
node_capacities, transport_capacities, demands, movements_orig = init_simulation(num_nodes, num_days)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

movements = movements_orig.ravel()

Randomly generated
1 days, 2 hospitals
ICU capacities of hospitals: [277, 154]
Daily transport capacities: [20, 22], 
Daily simulated demand (from uniform distrution): 
[[81], [81]]
